# Scraping subreddits


## Problem statement
---
We are producing a trivia focused, _Jeopardy_ or _Who Wants to Be a Millionaire_* style game show, where we want the audience to guess the source of the movie details. Everything is scrambled together! The task: figure out if the details came from good movie details, or whether the movie production team took a shorcut and landed in Sh**ty Movie Details!

*Trademarks of their appropiate productions

To do this, we are going to solve the problem using a classifer trained on some subreddit data.

_Thanks to Gwen Rathgeber for some inspiration during the quest for a compelling problem statement!_

![](https://www.bigraildiversity.co.uk/wp-content/uploads/2018/10/Night-at-the-Movies-Converted-900x600.png)


## Data sources and references

---

1. GENERAL NOTE ON ATTRIBUTION: The work throughout this Report, including many if not most coding techniques, rely and borrow heavily from code discussed in Riley Dallas, Sophie "Sonya" Tabac, Charlie Rice, Heather Robbins, and Gwen Rathgeber's class lectures, Notebooks, GitHub repositories, tips on techniques and troubleshooting help. The code is adapted to solve our specific problem.


2. The following subReddits were scraped:

    * [Movie Details](https://www.reddit.com/r/MovieDetails/)
    * [Sh**ty Movie Details](https://www.reddit.com/r/shittymoviedetails/)

---

### Note on the data and style

Strong language may appear in various Reddit posts in raw form. To the extent possible, it shall be cleaned in the course of the project. There also may be some humor used throughout the presentation of the analysis.

---

## Webscrape - let's fetch some data!

We'll import all our required libraries up here.

In [1]:
#import libraries
import pandas as pd, numpy as np, requests, time, nltk, datetime as dt

#NLP
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import gensim.downloader as api #allows us to get word2vec anf glove embeddins that we need
from gensim.models.word2vec import Word2Vec
from transformers import pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


#classifiers
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, confusion_matrix, plot_confusion_matrix, \
                             recall_score, precision_score)
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.dummy import DummyClassifier


# easier to see full text with a bigger maxwidth:
pd.options.display.max_colwidth = 200

In [2]:
#pip install python-Levenshtein
#this is for the gensim.similarities.levenshtein submodule

Next, we will use the PushShift API to collect the subReddit data we need.

More information about the API can be found on the [Pushshift repo](https://github.com/pushshift/api).

### Set up single post scrape

In [3]:
#Query the PushShift API for the subReddit data we need
url = 'https://api.pushshift.io/reddit/search/submission?subreddit=MovieDetails'
url2 = 'https://api.pushshift.io/reddit/search/submission?subreddit=shittymoviedetails'

_PRO-TIP: We can paste the URL in browser to preview the JSON format._

Next, we need to define some parameters and then confirm that we get a response from the API, from both the subReddits. (NOTE: There is no key needed to access the Reddit API.)

In [4]:
#define what we need to get from the subreddit
params = {
    'subreddit': 'MovieDetails',
    'size' : 100 #100 seems to be the max, even if change this to a greater size
#    'before': ''
}

In [5]:
#define what we need to get from the subreddit
params2 = {
    'subreddit': 'shittymoviedetails',
    'size' : 100 #100 seems to be the max, even if change this to a greater size
#    'before': ''
}

Let's check that our response is valid -- we are looking for a 200 response code.

In [6]:
res = requests.get(url, params) 
res.status_code

200

In [7]:
res2 = requests.get(url2, params2)
res2.status_code

200

Let's look at the content we fetched, for a single post:

In [8]:
print(type(res)) #check attrs

data = res.json() #get the content in JSON format

orig_posts = data['data'][0] #Fetch the list of first posts

#verify what we got
print(orig_posts)

print(type(orig_posts))

print(data.keys())

<class 'requests.models.Response'>
{'all_awardings': [], 'allow_live_comments': False, 'author': 'stalwartGRUNT', 'author_flair_css_class': None, 'author_flair_richtext': [], 'author_flair_text': None, 'author_flair_type': 'text', 'author_fullname': 't2_1k03g353', 'author_patreon_flair': False, 'author_premium': False, 'awarders': [], 'can_mod_post': False, 'contest_mode': False, 'created_utc': 1619331027, 'domain': 'i.redd.it', 'full_link': 'https://www.reddit.com/r/MovieDetails/comments/my298f/in_rango_2011_the_bottle_he_uses_to_shield/', 'gildings': {}, 'id': 'my298f', 'is_crosspostable': True, 'is_meta': False, 'is_original_content': False, 'is_reddit_media_domain': True, 'is_robot_indexable': True, 'is_self': False, 'is_video': False, 'link_flair_background_color': '#0079d3', 'link_flair_css_class': 'Image', 'link_flair_richtext': [{'e': 'text', 't': '🥚 Easter Egg'}], 'link_flair_template_id': 'b968d6fa-5f2d-11e7-8e64-0e8c19beb1c6', 'link_flair_text': '🥚 Easter Egg', 'link_flair_t

Everything we do, we have to repeat for our second subReddit.

In [9]:
#repeat for the 2nd subreddit
data2 = res2.json() #get the content in JSON format
posts2 = data2['data'][0] #Fetch the list of first post
posts2

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'ConsistentTop954',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_br0rfjjh',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1619332362,
 'domain': 'i.redd.it',
 'full_link': 'https://www.reddit.com/r/shittymoviedetails/comments/my2joe/in_joker_2019_before_the_show_starts_murray_tells/',
 'gildings': {},
 'id': 'my2joe',
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': True,
 'is_robot_indexable': True,
 'is_self': False,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 0,
 'num_crossposts': 0,
 'over_18': False

In [10]:
len(orig_posts)

66

Let's take a look at what we pulled in a more visually accessible format.

In [11]:
#create df
results_df = pd.DataFrame(data['data'])
results_df2 = pd.DataFrame(data2['data'])
results_df.head()

,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,author_premium,...,removed_by_category,media,media_embed,secure_media,secure_media_embed,gallery_data,is_gallery,media_metadata,author_flair_template_id,author_flair_text_color
0,[],False,stalwartGRUNT,None,[],None,text,t2_1k03g353,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[],False,WhirledNews,None,[],None,text,t2_3cbl0,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[],False,Qbccd,None,[],None,text,t2_7bllpt0q,False,False,...,moderator,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[],False,Twisted_Schwartz_,None,[],None,text,t2_266l79g,False,False,...,NaN,"{'oembed': {'author_name': 'Movieclips', 'author_url': 'https://www.youtube.com/c/MOVIECLIPS', 'height': 200, 'html': '&lt;iframe width=""356"" height=""200"" src=""https://www.youtube.com/embed/bBaNdH...","{'content': '&lt;iframe width=""356"" height=""200"" src=""https://www.youtube.com/embed/bBaNdHqC_Yo?start=9&amp;feature=oembed&amp;enablejsapi=1"" frameborder=""0"" allow=""accelerometer; autoplay; clipbo...","{'oembed': {'author_name': 'Movieclips', 'author_url': 'https://www.youtube.com/c/MOVIECLIPS', 'height': 200, 'html': '&lt;iframe width=""356"" height=""200"" src=""https://www.youtube.com/embed/bBaNdH...","{'content': '&lt;iframe width=""356"" height=""200"" src=""https://www.youtube.com/embed/bBaNdHqC_Yo?start=9&amp;feature=oembed&amp;enablejsapi=1"" frameborder=""0"" allow=""accelerometer; autoplay; clipbo...",NaN,NaN,NaN,NaN,NaN
4,[],False,Mysanthrop,None,[],None,text,t2_62tsl3o6,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
results_df.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
       'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_crosspostable', 'is_meta', 'is_original_content',
       'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
       'link_flair_background_color', 'link_flair_css_class',
       'link_flair_richtext', 'link_flair_template_id', 'link_flair_text',
       'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
       'no_follow', 'num_comments', 'num_crossposts', 'over_18',
       'parent_whitelist_status', 'permalink', 'pinned', 'post_hint',
       'preview', 'pwls', 'retrieved_on', 'score', 'selftext', 'send_replies',
       'spoiler', 'stickied', 'subreddit', 'subreddit_id',
       'subreddit_subscribers', 'subr

There is a lot of metadata here. We probably will not need most of it.

Here are the important tags we plan to use:

* subreddit
* selftext
* title
* created_utc
* author
* is_self (to filter out link-only posts)
* score
* num_comments

Let's grab just the metadata we need.

In [13]:
subfields = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'is_self', \
'score', 'num_comments']

In [14]:
results_df = results_df[subfields]
results_df2 = results_df2[subfields]
results_df.head(2)

,title,selftext,subreddit,created_utc,author,is_self,score,num_comments
0,"In Rango (2011), The bottle he uses to shield himself from the hawk has a faded Mexican soda company logo, jarritos",,MovieDetails,1619331027,stalwartGRUNT,False,1,1
1,Mortal Kombat 2021 - Tanya vs Kano,"In the final fight scene between the two, Tanya gets a knife knocked of her hand and it hits the bullseye on a dartboard with a picture on it. In that picture is a sketch of Tanya and Kano, in whi...",MovieDetails,1619329382,WhirledNews,True,1,1


Let's remove any duplicate posts.

In [15]:
#dupes
results_df.drop_duplicates(inplace=True)
results_df2.drop_duplicates(inplace=True)

We also only want original text content here.

In [16]:
#filter only for self posts
results_df = results_df[results_df["is_self"] == True]
results_df2 = results_df2[results_df2['is_self']==True]

We'll grab the timestamp so that we can set before and after the post we want, to pull in our desired volume of posts.

In [17]:
#convert timestamp to a format we understand
results_df['timestamp'] = results_df['created_utc'].map(dt.date.fromtimestamp)
results_df2['timestamp'] = results_df2['created_utc'].map(dt.date.fromtimestamp)
results_df['timestamp']

1     2021-04-24
2     2021-04-24
4     2021-04-24
15    2021-04-24
22    2021-04-24
23    2021-04-24
46    2021-04-24
59    2021-04-23
60    2021-04-23
61    2021-04-23
62    2021-04-23
64    2021-04-23
68    2021-04-23
84    2021-04-23
91    2021-04-23
94    2021-04-23
96    2021-04-23
97    2021-04-23
Name: timestamp, dtype: object

In [18]:
results_df.shape

(18, 9)

Looks like we get a couple days' worth in a single pull.

### Set up to pull @certain freq

Let's set up our pull to be a bit more dynamic.

In [19]:
#define endpoint
base_url = 'https://api.pushshift.io/reddit/search/submission'

#update params
subreddit = 'MovieDetails'
size = 100

#construct URL
stem = f'{base_url}?subreddit={subreddit}&size={size}'
stem

'https://api.pushshift.io/reddit/search/submission?subreddit=MovieDetails&size=100'

In [20]:
stem == url

False

Let's add in a time component to loop over multiple days' worth of posts.

In [21]:
days = 30
url = f'{stem}&after={days}d'
url

'https://api.pushshift.io/reddit/search/submission?subreddit=MovieDetails&size=100&after=30d'

In [22]:
#verify output

res=requests.get(url)
assert res.status_code == 200
json_data = res.json()

results_df = pd.DataFrame(json_data['data'])[subfields]

results_df[
    'created_utc'].map(dt.date.fromtimestamp).head()

0    2021-03-27
1    2021-03-27
2    2021-03-27
3    2021-03-27
4    2021-03-27
Name: created_utc, dtype: object

We see something from as far back as a month ago, which is what we were expecting.

In [23]:
results_df.shape

(100, 8)

### Automate pull

Let's set up a function that fetches posts and appends them to a dataframe.

We will then run this on our desired subReddits.

In [24]:
#loop to iterate through the pulls

def fetch_posts(subreddit, #specify subreddit
                kind = 'submission', #grab post
               day_window = 1000, n=5): #number of iterations to run
    #through; this is also how many dfs are output


#est. params
    
    
    base_url = f'https://api.pushshift.io/reddit/search/{kind}'

    stem = f'{base_url}?subreddit={subreddit}&size=500'

  
    posts = []

    for i in range(1, n+1): #that many iterations
        
        URL = '{}&after={}d'.format(stem, day_window *i)
        
        print('Query from timeframe: ' + URL)
        
        res = requests.get(URL)
        assert res.status_code == 200
        
        json_dict = res.json()['data'] #grab the single key from the list of json dicts
   
        df = pd.DataFrame.from_dict(json_dict) #convert dictionary info to a df
        
        posts.append(df) #append the df from each iteration to our posts list
        
        total_scraped = sum(len(x) for x in posts) #understand, how many posts we
        #are getting
            
        print(total_scraped)
        
        if total_scraped > 7500:
            break
        
        time.sleep(1) #wait 1 s between requests
        
    #merge list of dfs from our requests
    full_df = pd.concat(posts, sort=False)
    
  #  if kind == 'submission':
    
    full_df = full_df[subfields] #only want the specific columns
    full_df.drop_duplicates(inplace=True) #de-dupe posts
    full_df = full_df.loc[full_df['is_self']==True] #only grab original submissions
    
    full_df['timestamp'] = full_df['created_utc'].map(dt.date.fromtimestamp) #convert time
    
    return full_df.reset_index(drop=True)

In [25]:
movie_deets = fetch_posts('MovieDetails')
movie_deets.head()

Query from timeframe: https://api.pushshift.io/reddit/search/submission?subreddit=MovieDetails&size=500&after=1000d
100
Query from timeframe: https://api.pushshift.io/reddit/search/submission?subreddit=MovieDetails&size=500&after=2000d
200
Query from timeframe: https://api.pushshift.io/reddit/search/submission?subreddit=MovieDetails&size=500&after=3000d
300
Query from timeframe: https://api.pushshift.io/reddit/search/submission?subreddit=MovieDetails&size=500&after=4000d
400
Query from timeframe: https://api.pushshift.io/reddit/search/submission?subreddit=MovieDetails&size=500&after=5000d
500


,title,selftext,subreddit,created_utc,author,is_self,score,num_comments,timestamp
0,"In the dark knight rises when lucius fox show the new material to Bruce Wayne, he says that he keeps them there to ""keep them from falling into the wrong hands"". Later, the material literally fall...",https://i.redd.it/qvgqz019m1d11.png,MovieDetails,1532938057,sylvain147,True,1,0,2018-07-30
1,It's the end of the weekend.,[removed],MovieDetails,1532944839,AutoModerator,True,1,0,2018-07-30
2,Avengers Infinity War (2018) | Full Movie Free Download | WEB - DL | Direct Link And Torrent Links,[removed],MovieDetails,1532966211,lahirulg,True,1,0,2018-07-30
3,"In Spider-Man: Homecoming, the Vulture scavenges Chitauri tech from downed ships in the same way that a vulture scavenges meat from carcasses.",,MovieDetails,1532967088,dewisri,True,0,5,2018-07-30
4,"In Deadpool 1 and 2, while powerless, Wade never speaks directly to the audience, suggesting that his ability to see the 4th wall is another part of his mutation.","In Deadpool 1, it is only the ""current"" Deadpool providing narration. During the various flashback scenes, Powerless Wade never talks directly to the audience. He does make 4th wall breaking jokes...",MovieDetails,1532973519,Grapz224,True,12585,240,2018-07-30


In [26]:
type(movie_deets)

pandas.core.frame.DataFrame

In [27]:
movie_deets.columns

Index(['title', 'selftext', 'subreddit', 'created_utc', 'author', 'is_self',
       'score', 'num_comments', 'timestamp'],
      dtype='object')

In [28]:
movie_deets.shape

(40, 9)

### Repeat this process for our second subreddit

In [29]:
bad_deets = fetch_posts('shittymoviedetails')
print(bad_deets.shape)
bad_deets.head()

Query from timeframe: https://api.pushshift.io/reddit/search/submission?subreddit=shittymoviedetails&size=500&after=1000d
100
Query from timeframe: https://api.pushshift.io/reddit/search/submission?subreddit=shittymoviedetails&size=500&after=2000d
200
Query from timeframe: https://api.pushshift.io/reddit/search/submission?subreddit=shittymoviedetails&size=500&after=3000d
300
Query from timeframe: https://api.pushshift.io/reddit/search/submission?subreddit=shittymoviedetails&size=500&after=4000d
400
Query from timeframe: https://api.pushshift.io/reddit/search/submission?subreddit=shittymoviedetails&size=500&after=5000d
500
(26, 9)


,title,selftext,subreddit,created_utc,author,is_self,score,num_comments,timestamp
0,"Star Wars uses frequent subtle references to fascism for the Empire's aesthetic. For example, Star Destroyers are shaped like pizza slices. Benito Mussolini, a famous fascist, was Italian.",https://i.redd.it/79e5b9iy75d11.jpg,shittymoviedetails,1532981670,Chikalaka,True,296,5,2018-07-30
1,"In the Star Wars, you can see Luke Skywalker shoot some missiles into the Death Star before it blows up",,shittymoviedetails,1533149696,psxv,True,38,3,2018-08-01
2,"The latest Avengers movie is called Infinity War, however his title is misleading as it did not go for infinity, but rather 2 hours and 30 minutes.",Truly the scam of the century.,shittymoviedetails,1533169538,NDOperator,True,19,0,2018-08-01
3,"In Baby Driver, Doc (kevin spacey) says 'i was blinded by the balls on that kid', this references/ foreshadows the actors sexual attraction to BOYS, and the allegations he would face",https://i.redd.it/n9ycz2dw8pd11.jpg,shittymoviedetails,1533224098,Thatsabigvoid,True,16,0,2018-08-02
4,The movie 'In Bruges' is aptly named because thr majority of the film takes place in Bruges that's in Belgium.,,shittymoviedetails,1533308375,Ralph-Hinkley,True,23,1,2018-08-03


### Output the dataframes!

In [31]:
movie_deets.to_csv('../data_outputs/good_movie_deets.csv')
bad_deets.to_csv('../data_outputs/bad_movie_deets.csv')

## Cleaning and EDA

Let's look at basic stats about our data.

In [ ]:
#<df>.isnull().sum()

Let's understand the density and usefulness of our content, by analyzing the volume of comments and looking for the richness of the text-heavy columns we are expecting.

We are also going to see, whether there are any interesting prospective features for training our models.

In [ ]:
#<df>['num_comments'].value_counts()

In [ ]:
#check for post density
#<df>['selftext'].value_counts()

In [ ]:
#<df>['title'].value_counts()

In [ ]:
#<df>['author'].value_counts() #get only unique

## NLP & feature eng

### Word analysis

In [ ]:
#word freq
#vectorize self-text

In [ ]:
#sentiment